# STAT GR5702 Exploratory Data Analysis & Visualisation - Community Contribution.
# Using matplotlib in Python for Function and Histogram Animation

Name: James Franco

UNI: jaf2249

Section: 002

## Introduction

Across many fields of science (computational science, data science, etc), matplotlib in Python is used as a static visualisation tool. i.e., not animated. But what a lot of people don't know is that we can also use matplotlib for animations! This is very useful for many applications that aren't fully explainable via static plots; such as plotting numerical solutions to time-dependent PDEs, or showing how data histograms change over time. The matplotlib framework, while very robust and flexible, can be quite hard to grasp without seeing it used in practice. For example, knowing what the inner workings of a Rectangle or a Line2D object can be very helpful, but won't necessarily help you in using matplotlib for effective data visualisation. I'd like this notebook to be a couple of (simple) examples showing how matplotlib can be used to create animations that can be easily copied and/or extended for other uses. Alongside that, I'll briefly explain how things work along the way in order to give a high-level overview of what's going on, but will link the appropriate documentation for more in-depth reading. The notebook will be split up into 3 relevant sections:
1. Using FuncAnimation to animate functions on a set of axes
2. Using FuncAnimation to animate histograms on a set of axes
3. Combining these steps to do both at once, and animate histograms with extra axis information

## Imports

We'll need to import matplotlib's animation package, and also HTML from IPython.display in order to show animations in the Jupyter environment.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

## Function Animations

For purposes of testing, we'll plot the function $f(x,t) = \sin(x-t)$, for time values $t \in [0,2\pi]$, across the domain $x \in [0,2\pi]$. We will animate the following aspects of the graph:
* The sin function itself
* The legend containing the label of the sin function
* The title of the graph


Broadly speaking, in pseudo-pseudo-code, the animation process is as follows.
* Set up matplotlib figure and axis
* Plot any details on the axes you'd like (e.g. function data, axis labels, title, etc)
    * Make sure to name any variables you'd like to change during the animation process, e.g. title = ax.set_title(...) if you'd like the title to be updated each new animation frame, but you can just call ax.set_xlabel(...) normally without naming it if you wouldn't like the x-axis label to change during the animation
* Set up animate(...) function which prescribes how the figure should be changed with each new frame in the animation (more on this later)
* Set ani = animation.FuncAnimation(...) which renders the animation (more on this later)
* (If displaying in IPython notebook) call HTML(ani.to_html5_video())

### animate() function info

animate() is the user-defined function which, when given a frame number $i$, defines what parts of the graph$^1$ should be updated at frame $i$. To do this, the variables saved during the setup process should be changed in-function, and then returned as an iterable at the end. See below for implementation.


$^1$ Each "part" of the graph is an Artist object in matplotlib, which is a base class (see: https://matplotlib.org/stable/api/artist_api.html for more info and inheritance diagrams) that essentially means "something that can be put on the axis". This can include Line2D (object returned by ax.plot()), Legend, etc.

### FuncAnimation() function info

FuncAnimation() is the function which, given a figure (which we initially set up), an animate() function, and some other kwargs about animation details, will render the animation and save it as an object, which is typically named 'ani'. See https://matplotlib.org/stable/api/_as_gen/matplotlib.animation.FuncAnimation.html#matplotlib.animation.FuncAnimation for documentation. Some important kwargs are:
* frames: if given an int, then is the number of frames in the animation (can pass other types too, see docs)
* interval (float): number of milliseconds that each frame lasts for
* repeat (bool): True if you'd like the animation to repeat when it's done, False if you want it to cease after 1 run through

### The above in practice

In [2]:
# Defining sin wave function
f = lambda x,t : np.sin(x - t)

# Discretising time t in [0,2pi] and space x in [0,2pi]
num_x_points = 200
x = np.linspace(0,2*np.pi, num_x_points)
t_0 = 0
t_f = 2 * np.pi
num_timesteps = 200
t = np.linspace(t_0, t_f, num_timesteps)
dt = t[1] - t[0]

# Filling 2D array with row i being sin(x-t) at time t_i 
func_data = np.zeros((num_timesteps, num_x_points))
for i in range(num_timesteps):
    func_data[i] = f(x,t[i])

In [3]:
def animate(i):
    line.set_ydata(func_data[i]) # update sin wave data
    line.set_label(r"$\sin(x - {{{:.2f}}})$".format(t[i])) # update sin wave label
    legend = ax.legend(loc="upper right") # update legend
    title.set_text("Function Animation Example! (Frame {}/{})".format(i+1, num_timesteps)) # update title
    return [line, legend, title] # return all updated Artist objects

In [4]:
# Set up figure and axes
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(1,1,1)

# Initial to-be-animated plot details
line, = ax.plot(x, func_data[0], label=r"$\sin(x - {{{:.2f}}})$".format(t_0))
legend = ax.legend(loc="upper right")
title = ax.set_title("Function Animation Example! (Frame 1/{})".format(num_timesteps), fontsize=18)

# Static plot details
ax.set_xlabel("x", fontsize=16)
ax.set_ylabel("y", fontsize=16)
ax.grid(True)

# Create animation
ani = animation.FuncAnimation(fig, animate, frames=num_timesteps, interval = dt*1000, repeat=False, blit=True)

# close the figure to not plot anything
plt.close(fig)

In [5]:
# Render animation as html5 video for displaying in Jupyter notebook
HTML(ani.to_html5_video())

## Histogram Animations

There is one key difference with histogram animations than the above simple function plots. That is before, we were able to simply change the data of the Line2D object. However here, this is not the case. On a high level, the way we have to get around this is quite finicky. Instead, a histogram is a collection of Rectangle objects, of which we must calculate the height of at each iteration step using numpy, and then change the height of each Rectangle. This process is shown in the docs here: https://matplotlib.org/stable/gallery/animation/animated_histogram.html and requires a wrapper function around animate(), which we will call prepare_animation(). The reason we need this wrapper function is because animate() must take in only the frame_number i as its input,

To illustrate this in practice, we'll slowly transform 100,000 random samples of a Gaussian $\sim \mathcal{N}(0,2^2)$ into a Gaussian $\sim \mathcal{N}(3, 4^2)$.

In [6]:
# Histogram setup
num_histograms = 150
num_points = 100000
all_data = np.zeros((num_histograms, num_points))
start_mean = 0
end_mean = 3
start_std = 2
end_std = 4

# Populate 2D array, with row i as 100,000 samples from N(mu_i, sigma^2_i)
for i in range(num_histograms):
    mu = (1 - (num_histograms-i)/(num_histograms))*end_mean + (num_histograms-i)/(num_histograms) * start_mean
    sigma = (1 - (num_histograms-i)/(num_histograms))*end_std + (num_histograms-i)/(num_histograms) * start_std
    all_data[i] = np.random.normal(loc=mu, scale=sigma, size=num_points)

# Creating histogram bins
minbin,maxbin,nbins = -15, 15, 100
bins = np.linspace(minbin, maxbin, nbins)

In [7]:
# Get histogram heights of our data with numpy
data = all_data[0]
n, _ = np.histogram(data, bins, density=True)

# Wrapper function to take in bar_container, which is list of Rectangles
def prepare_animation(bar_container):
    # Actual animation function
    def animate(frame_number):
        # Get new histogram with same bins
        data = all_data[frame_number]
        n, _ = np.histogram(data, bins, density=True)
        # Update the height of each Rectangle (histogram bar)
        for count, rect in zip(n, bar_container.patches):
            rect.set_height(count)
        return bar_container.patches
    return animate

In [8]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(1,1,1)
ax.grid(True)
ax.set_ylim(top=0.25)

_, _, bar_container = ax.hist(data, bins, lw=1,
                              ec="gray", fc="cyan", alpha=0.8, density=True)


ani2 = animation.FuncAnimation(fig, prepare_animation(bar_container), frames=num_histograms, interval = 40,
                              repeat=False, blit=True)
plt.close(fig)

In [9]:
HTML(ani2.to_html5_video())

## Combining Functionalities (Histogram + Updated Title + Mean)

Since there is an extra wrapper function required to handle updating histogram Rectangles, one might ask how to update other information in the animated graph alongside histogram information. This section will show you how to do that. Similarly to section 1 - all we must do is update the animate() function, except this time it's nested within prepare_animation(). 

Say for example, we would like to keep track of the mean of the underlying distribution at each time step, and plot it as a vertical dotted line that updates with time. We'd also like to add a legend to the plot, that contains the label $\mu = \mu_i$, where $\mu_i$ is the mean at histogram number $i$. We'll also continually update the title to say "Histogram Number $i$".

We must first set up the figure and plot the initial instances of all of these attributes we like to plot. We'll save them under variable names in order to update them within the animate() function.

In [10]:
# Get Initial Data
data = all_data[0]
# Histogram Info
n, _ = np.histogram(data, bins, density=True)
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(1,1,1)
ax.grid(True)
ymax=0.25
# Histogram Rectangles
_, _, bar_container = ax.hist(data, bins, lw=1, ec="gray", fc="cyan", alpha=0.8, density=True)
# Title 
title = ax.set_title("Histogram Number {}".format(1), fontsize=18)
# Mean dotted line
mean_line = ax.axvline(x=start_mean, ymin=0, ymax=1, label=r"$\mu = ${:.3f}".format(start_mean), lw=2, ls="--")
# Legend
legend = ax.legend(loc="best")
# Static (non-animated) axis attributes
ax.set_ylim(top=ymax)
ax.set_ylabel("Density", fontsize=16)
ax.set_xlabel("x", fontsize=16)

plt.close(fig)

Now, we may change the animate() function to update and return all the named axis attributes we've defined above. It's important to remember that we must return a sequence of Artist objects (Line, Rectangle, Text, etc...). Since bar_container.patches is already a list of Rectangles, we can put the other objects into a list and append it onto bar_container.patches for the return statement.

In [11]:
def prepare_animation(bar_container):

    def animate(frame_number):
        # Get new histogram with same bins
        data = all_data[frame_number]
        n, _ = np.histogram(data, bins, density=True)
        # Update the height of each Rectangle (histogram bar)
        for count, rect in zip(n, bar_container.patches):
            rect.set_height(count)
        title.set_text("Histogram Number {}".format(frame_number+1))
        mu = (1 - (num_histograms-frame_number)/(num_histograms))*end_mean + (num_histograms-frame_number)/(num_histograms) * start_mean
        mean_line.set_xdata([mu,mu])
        mean_line.set_label(r"$\mu = ${:.3f}".format(mu))
        legend = ax.legend(loc="best")

        return bar_container.patches + [mean_line, title, legend]
    return animate

Now we can call animation.FuncAnimation() as we did before.

In [12]:
ani3 = animation.FuncAnimation(fig, prepare_animation(bar_container), frames=num_histograms, interval = 40,
                              repeat=False, blit=True)
plt.close(fig)

In [13]:
HTML(ani3.to_html5_video())